# RAG Prompt Compression

- RAG에서 검색된 문서 컨텍스트를 LLM에 넣기 전에 압축(요약/추출)해서 프롬프트를 가볍게 만드는 단계(또는 그 함수/체인)”
    - rag: Retrieval-Augmented Generation(검색 기반 생성)
    - prompt: LLM에 넣는 입력(프롬프트/컨텍스트)
    - compression: 길이를 줄이거나 핵심만 남기는 처리(요약, 중요 문장 추출, 중복 제거 등)

- rag_prompt_compression의 목적
    - 토큰 제한(컨텍스트 윈도우) 안에 더 많은 핵심정보를 넣기
    - 비용/지연 줄이기(입력 토큰 감소)
    - 불필요한 문장 제거로 답변 품질 안정화

In [2]:
%pip install langchain langchain-openai -Uq

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGSMITH_API_KEY'] = os.getenv('langsmith_key')
os.environ['LANGSMITH_PROJECT'] = 'skn23-langchain'
os.environ['OPENAI_API_KEY'] = os.getenv("openai_key")

In [4]:
from langchain_core.documents import Document  # LangChain Document 클래스 import

# 벡터DB에서 검색된 것처럼 Document 리스트를 반환하는 더미 리트리버 함수
def retrieve_vectordb(query=None):
    return [  # Document 객체 리스트 반환
        Document(  # 1번 문서 생성
            page_content="""현대 교육 분야에서는 인공지능(AI) 기반 플랫폼이 단순한 보조 도구를 넘어 학습 전반을 설계하는 핵심 인프라로 자리 잡고 있다. 초기 AI 교육 시스템은 출결 관리, 문제 자동 채점, 진도 체크처럼 교사의 업무 부담을 줄이는 기능에 집중했으나, 최근에는 학습자의 행동 로그, 성취도, 반복 오류 패턴 등을 분석해 개인별 학습 성향을 파악하는 방향으로 발전했다. 이를 통해 학습자마다 난이도, 콘텐츠 유형, 학습 속도를 다르게 조정하는 맞춤형 학습 추천 시스템이 구현되고 있다. 더 나아가 자연어 처리(NLP) 기술의 발전으로 강의 영상이나 교재를 자동 요약하거나 핵심 개념을 재구성해 제공하는 기능이 가능해졌으며, 학습자가 질문을 입력하면 실시간으로 맥락을 이해하고 답변하는 챗봇 튜터가 등장해 자기주도 학습을 강화하고 있다. 이러한 변화는 교육을 일방향 전달에서 상호작용 기반 학습으로 전환시키는 중요한 역할을 하고 있다.""",  # 문서 본문(교육/AI 사례)
            metadata={"source": "doc1"}  # 문서 출처 메타데이터
        ),
        Document(  # 2번 문서 생성
            page_content="""Retrieval-Augmented Generation(RAG)은 대규모 언어 모델(LLM)이 가진 언어 생성 능력에 외부 지식 저장소를 결합함으로써, 모델이 학습하지 않았거나 최신성이 필요한 정보까지 활용할 수 있도록 한 구조이다. 기존 LLM은 학습 시점 이후의 정보를 반영하기 어렵고, 사실과 다른 내용을 생성하는 문제가 있었으나, RAG는 벡터 DB에 저장된 문서를 검색해 이를 보완한다. RAG 파이프라인은 일반적으로 세 단계로 구성되며, 먼저 사용자의 질의를 임베딩 벡터로 변환해 의미적으로 가장 유사한 문서를 Top-K 방식으로 검색한다. 이후 검색된 문서들을 프롬프트 컨텍스트로 구성해 LLM에 전달하고, 마지막으로 LLM이 해당 정보를 종합·추론해 최종 답변을 생성한다. 이 구조를 통해 답변의 정확성과 근거성이 향상되며, 기업 문서 검색, 사내 지식 Q&A, 교육용 튜터 시스템 등 다양한 실무 환경에서 활용되고 있다.""",  # 문서 본문(RAG 설명)
            metadata={"source": "doc2"}  # 문서 출처 메타데이터
        ),
        Document(  # 3번 문서 생성
            page_content="""프롬프트 압축(Prompt Compression)은 LLM에 전달되는 입력 컨텍스트를 최대한 간결하게 정제하여, 제한된 토큰 수 안에서 핵심 정보만 효과적으로 전달하기 위한 기법이다. LLM은 입력 토큰 수에 따라 비용과 응답 시간이 증가하기 때문에, 불필요한 문장이나 중복된 설명이 많을수록 시스템 효율이 급격히 떨어진다. 이를 해결하기 위해 프롬프트 압축은 문서 요약, 중요 문장 추출, 의미 중복 제거, 구조화된 키포인트 변환 등의 방법을 사용한다. 특히 RAG 환경에서는 검색된 여러 문서를 그대로 넣는 대신 핵심 정보만 압축해 전달함으로써, 답변 품질을 유지하면서도 비용과 지연을 줄일 수 있다. 결과적으로 프롬프트 압축은 대규모 LLM 서비스를 안정적으로 운영하기 위한 필수 최적화 전략으로 활용되고 있다.""",  # 문서 본문(프롬프트 압축 설명)
            metadata={"source": "doc3"}  # 문서 출처 메타데이터
        )
    ]

retrieve_vectordb()  # 함수 실행(문서 리스트 생성)

[Document(metadata={'source': 'doc1'}, page_content='현대 교육 분야에서는 인공지능(AI) 기반 플랫폼이 단순한 보조 도구를 넘어 학습 전반을 설계하는 핵심 인프라로 자리 잡고 있다. 초기 AI 교육 시스템은 출결 관리, 문제 자동 채점, 진도 체크처럼 교사의 업무 부담을 줄이는 기능에 집중했으나, 최근에는 학습자의 행동 로그, 성취도, 반복 오류 패턴 등을 분석해 개인별 학습 성향을 파악하는 방향으로 발전했다. 이를 통해 학습자마다 난이도, 콘텐츠 유형, 학습 속도를 다르게 조정하는 맞춤형 학습 추천 시스템이 구현되고 있다. 더 나아가 자연어 처리(NLP) 기술의 발전으로 강의 영상이나 교재를 자동 요약하거나 핵심 개념을 재구성해 제공하는 기능이 가능해졌으며, 학습자가 질문을 입력하면 실시간으로 맥락을 이해하고 답변하는 챗봇 튜터가 등장해 자기주도 학습을 강화하고 있다. 이러한 변화는 교육을 일방향 전달에서 상호작용 기반 학습으로 전환시키는 중요한 역할을 하고 있다.'),
 Document(metadata={'source': 'doc2'}, page_content='Retrieval-Augmented Generation(RAG)은 대규모 언어 모델(LLM)이 가진 언어 생성 능력에 외부 지식 저장소를 결합함으로써, 모델이 학습하지 않았거나 최신성이 필요한 정보까지 활용할 수 있도록 한 구조이다. 기존 LLM은 학습 시점 이후의 정보를 반영하기 어렵고, 사실과 다른 내용을 생성하는 문제가 있었으나, RAG는 벡터 DB에 저장된 문서를 검색해 이를 보완한다. RAG 파이프라인은 일반적으로 세 단계로 구성되며, 먼저 사용자의 질의를 임베딩 벡터로 변환해 의미적으로 가장 유사한 문서를 Top-K 방식으로 검색한다. 이후 검색된 문서들을 프롬프트 컨텍스트로 구성해 LLM에 전달하고, 마지막으로 LLM이 해당 정보를 종합·추론해 최종 답변을 생성한다. 이 구조를 통해 답변의 정확성과 근거성이 향상되며, 기업 문서 검색, 사내 지

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import init_chat_model

prompt = PromptTemplate.from_template('''
다음 문서의 내용을 300토큰 이내로 요약해 주세요.

{docs}
''')

llm = init_chat_model('gpt-4.1-mini', temperature=0)  # 답변 생성용 LLM 설정 (창의성 0)
output_parser = StrOutputParser()      # LLM 출력 문자열로 파싱

summary_chain = prompt | llm | output_parser

retrieved_docs = retrieve_vectordb()  # 더미 벡터 검색 수행
context = '\n\n'.join([f'''
[{doc.metadata['source']}]
{doc.page_content}''' for doc in retrieved_docs])  # 문서들을 하나의 컨텍스트 문자열로 결합
print(context)

compressed_context = summary_chain.invoke({'docs': context})  # 컨텍스트를 300토큰 이내로 요약
print(f'\n\n압축된 컨텍스트: \n{compressed_context}')

c:\Users\Playdata\nlp\nlp_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



[doc1]
현대 교육 분야에서는 인공지능(AI) 기반 플랫폼이 단순한 보조 도구를 넘어 학습 전반을 설계하는 핵심 인프라로 자리 잡고 있다. 초기 AI 교육 시스템은 출결 관리, 문제 자동 채점, 진도 체크처럼 교사의 업무 부담을 줄이는 기능에 집중했으나, 최근에는 학습자의 행동 로그, 성취도, 반복 오류 패턴 등을 분석해 개인별 학습 성향을 파악하는 방향으로 발전했다. 이를 통해 학습자마다 난이도, 콘텐츠 유형, 학습 속도를 다르게 조정하는 맞춤형 학습 추천 시스템이 구현되고 있다. 더 나아가 자연어 처리(NLP) 기술의 발전으로 강의 영상이나 교재를 자동 요약하거나 핵심 개념을 재구성해 제공하는 기능이 가능해졌으며, 학습자가 질문을 입력하면 실시간으로 맥락을 이해하고 답변하는 챗봇 튜터가 등장해 자기주도 학습을 강화하고 있다. 이러한 변화는 교육을 일방향 전달에서 상호작용 기반 학습으로 전환시키는 중요한 역할을 하고 있다.


[doc2]
Retrieval-Augmented Generation(RAG)은 대규모 언어 모델(LLM)이 가진 언어 생성 능력에 외부 지식 저장소를 결합함으로써, 모델이 학습하지 않았거나 최신성이 필요한 정보까지 활용할 수 있도록 한 구조이다. 기존 LLM은 학습 시점 이후의 정보를 반영하기 어렵고, 사실과 다른 내용을 생성하는 문제가 있었으나, RAG는 벡터 DB에 저장된 문서를 검색해 이를 보완한다. RAG 파이프라인은 일반적으로 세 단계로 구성되며, 먼저 사용자의 질의를 임베딩 벡터로 변환해 의미적으로 가장 유사한 문서를 Top-K 방식으로 검색한다. 이후 검색된 문서들을 프롬프트 컨텍스트로 구성해 LLM에 전달하고, 마지막으로 LLM이 해당 정보를 종합·추론해 최종 답변을 생성한다. 이 구조를 통해 답변의 정확성과 근거성이 향상되며, 기업 문서 검색, 사내 지식 Q&A, 교육용 튜터 시스템 등 다양한 실무 환경에서 활용되고 있다.


[doc3]
프롬프트 압축(Prompt Compression)은 LLM에 전달되는 입력 

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import init_chat_model

prompt = PromptTemplate.from_template('''
[조회된 문서]에 기반해서 사용자의 [질문]에 성실하게 답변해 주세요.

[조회된 문서]
{context}

[질문]
{question}
''')

question = 'RAG가 뭔가요?'

llm = init_chat_model('gpt-4.1-mini')  # 답변 생성용 LLM 설정
output_parser = StrOutputParser()      # LLM 출력 문자열로 파싱

rag_chain = prompt | llm | output_parser

response = rag_chain.invoke({'context': compressed_context, 'question': question})  # 요약 컨텍스트 + 질문으로 답변 생성
print(response)

RAG(Retrieval-Augmented Generation)는 대규모 언어 모델(LLM)에 외부 지식 저장소를 결합해 최신 정보와 근거 있는 답변을 제공하는 기술입니다. 이는 문서 검색과 LLM의 추론 기능을 결합하여 답변의 정확성을 높이는 방식으로 작동합니다. 즉, 필요한 정보를 외부에서 검색한 후 이를 기반으로 언어 모델이 더욱 신뢰할 수 있는 결과를 생성하도록 돕는 기술이라고 할 수 있습니다.


In [8]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import init_chat_model

prompt = PromptTemplate.from_template('''
다음 문서의 내용을 300토큰 이내로 요약해 주세요.

{docs}
''')

# 요약 체인
summary_llm = init_chat_model('gpt-4.1-mini', temperature=0)  # 답변 생성용 LLM 설정 (창의성 0)
output_parser = StrOutputParser()      # LLM 출력 문자열로 파싱

summary_chain = prompt | summary_llm | output_parser

# Rag 체인
rag_llm = init_chat_model('openai:gpt-4.1-mini')
prompt = PromptTemplate.from_template('''
[조회된 문서]에 기반해서 사용자의 [질문]에 성실하게 답변해 주세요.

[조회된 문서]
{context}

[질문]
{question}
''')
rag_chain = prompt | rag_llm | output_parser

# 질문을 받아 벡터 DB 조회 -> 요약 -> 문서 기반 답변까지 한번에 수행하는 RAG 함수
def rag(question: str):
    retrieved_docs = retrieve_vectordb(question)  # 더미 벡터 검색 수행
    context = '\n\n'.join([f'''
    [{doc.metadata['source']}]
    {doc.page_content}''' for doc in retrieved_docs])  # 문서들을 하나의 컨텍스트 문자열로 결합
    compressed_context = summary_chain.invoke({'docs': context})  # 컨텍스트를 300토큰 이내로 요약

    return rag_chain.invoke({'context': compressed_context, 'question': question})  # 요약 컨텍스트 + 질문으로 답변 생성

question = 'RAG가 뭐에요?'
response = rag(question)
print(response)

RAG(Retrieval-Augmented Generation)는 대규모 언어 모델(LLM)에 외부 지식 저장소를 결합해 최신 정보와 근거 있는 답변을 제공하는 기술입니다. 사용자의 질문을 벡터 형태로 변환해 관련 문서를 검색하고, 검색된 문서를 바탕으로 LLM이 최종 답변을 생성하는 3단계 구조로 이루어져 있습니다. 이 방식은 특히 기업과 교육 분야에서 정확하고 신뢰할 수 있는 정보를 제공하는 데 활용됩니다.


- **사내 문서 검색형 챗봇**에서 가장 많이 사용되는 RAG 기본 패턴 실습 (규정, 매뉴얼, 기술 문서, 교육 자료 등)
- Prompt Compression을 적용하면 **문서량이 많아도 토큰 한도를 안정적으로 관리**할 수 있어, 대규모 서비스 운영 시 **비용 절감과 응답 속도 개선**에 직접적인 효과가 있다.
- 요약 → RAG 응답을 분리한 구조는  
  - 요약 캐싱  
  - 문서 업데이트 시 재요약  
  - 사용자 질문 유형별 요약 전략 분리  
  등 **운영 관점의 확장성**이 높다.
- 실제 실무에서는 이 파이프라인을  
  - 고객센터 FAQ 자동 응답  
  - 내부 지식 Q&A 시스템  
  - 교육/온보딩 튜터  
  - 법무·의료 문서 보조 분석  
  등에 그대로 적용할 수 있다.